In [2]:
import os
from PIL import Image
import torchvision.transforms.functional as TF
import numpy as np

# Rutas de entrada y salida
input_dir = r"C:\Users\jerij\Carpetas\Clases Formaciones Curro\Master\2Semestre\Proyecto\Data\Pruebas"  # Carpeta con imágenes y etiquetas
output_images_dir = r"C:\Users\jerij\Carpetas\Clases Formaciones Curro\Master\2Semestre\Proyecto\Data\Pruebas"
output_labels_dir = r"C:\Users\jerij\Carpetas\Clases Formaciones Curro\Master\2Semestre\Proyecto\Data\Pruebas" 
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Verificar si una etiqueta es NOK (contiene píxeles distintos de negro)
def is_nok(label_path):
    label = Image.open(label_path).convert("L")
    label_array = np.array(label)
    return np.any(label_array > 0)  # Verificar si hay píxeles distintos de 0

# Data augmentation sincronizado
def augment_image_and_label(image, label):
    augmented_pairs = []

    # Flip horizontal
    flipped_image = TF.hflip(image)
    flipped_label = TF.hflip(label)
    augmented_pairs.append((flipped_image.resize((512, 1024)), flipped_label.resize((512, 1024))))

    # Flip vertical
    flipped_image = TF.vflip(image)
    flipped_label = TF.vflip(label)
    augmented_pairs.append((flipped_image.resize((512, 1024)), flipped_label.resize((512, 1024))))

    # Rotación 180 grados
    rotated_image = image.rotate(180, expand=True)
    rotated_label = label.rotate(180, expand=True)
    augmented_pairs.append((rotated_image.resize((512, 1024)), rotated_label.resize((512, 1024))))

    # Traslaciones
    for dx, dy in [(20, 0), (-20, 0), (0, 20), (0, -20)]:
        translated_image = TF.affine(image, angle=0, translate=(dx, dy), scale=1.0, shear=0)
        translated_label = TF.affine(label, angle=0, translate=(dx, dy), scale=1.0, shear=0)
        augmented_pairs.append((translated_image.resize((512, 1024)), translated_label.resize((512, 1024))))

    return augmented_pairs

# Procesar todas las imágenes y etiquetas
file_list = sorted(os.listdir(input_dir))

for file_name in file_list:
    if file_name.endswith("_label.bmp"):  # Identificar etiquetas
        label_path = os.path.join(input_dir, file_name)
        image_path = label_path.replace("_label.bmp", ".jpg")  

        if os.path.exists(image_path) and is_nok(label_path):  # Solo procesar NOK
            # Cargar imagen y etiqueta
            image = Image.open(image_path).convert("L").resize((512, 1024))
            label = Image.open(label_path).convert("L").resize((512, 1024))

            # Aplicar data augmentation
            augmented_data = augment_image_and_label(image, label)

            # Guardar resultados
            base_name = os.path.splitext(file_name)[0].replace("_label", "")
            for idx, (aug_image, aug_label) in enumerate(augmented_data):
                aug_image.save(os.path.join(output_images_dir, f"{base_name}_aug_{idx}.jpg"))
                aug_label.save(os.path.join(output_labels_dir, f"{base_name}_aug_{idx}_label.bmp"))

print("Data augmentation hecho.")



Data augmentation hecho.
